#1.Lower casing


In [ ]:
df["text_lower"] = df["text"].str.lower()

# 2.removal of punctuations

Punctuation removal might be a good step, when punctuation does not brings additional value for text vectorization. Punctuation removal is better to be done after the tokenization step, doing it before might cause undesirable effects. Good choice for TF-IDF, Count, Binary vectorization.

In [ ]:
def remove_punctuation(text):
  no_punct="".join([c for c i text if c not in string.punctuation])
  return no_punct

In [ ]:
df['text']=df['text'].apply(lambda x: remove_punctuation(x))

or

In [ ]:
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df["text_wo_punct"] = df["text"].apply(lambda text: remove_punctuation(text))

#2.Removal of stopwords¶
Stopwords are commonly occuring words in a language like 'the', 'a' and so on. They can be removed from the text most of the times, as they don't provide valuable information for downstream analysis. In cases like Part of Speech tagging, we should not remove them as provide very valuable information about the POS.

These stopword lists are already compiled for different languages and we can safely use them. For example, the stopword list for english language from the nltk package can be seen below.

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [ ]:
#removing dem
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

df["text_wo_stop"] = df["text_wo_punct"].apply(lambda text: remove_stopwords(text))
df.head()

###second example

In [ ]:
def remove_stopwords(text):
  words=[w for w in text if w not in stopwords.words('english')]
  return words

  df['text']=df['text'].apply(lambda x:remove_stopwords(x))

#3.Removal of Frequent words
In the previos preprocessing step, we removed the stopwords based on language information. But say, if we have a domain specific corpus, we might also have some frequent words which are of not so much importance to us.

So this step is to remove the frequent words in the given corpus. If we use something like tfidf, this is automatically taken care of.

Let us get the most common words adn then remove them in the next step

In [ ]:
from collections import Counter
cnt = Counter()
for text in df["text_wo_stop"].values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(10)

In [ ]:
FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
def remove_freqwords(text):
    """custom function to remove the frequent words"""
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

df["text_wo_stopfreq"] = df["text_wo_stop"].apply(lambda text: remove_freqwords(text))
df.head()

#4.Removal of Rare words¶
This is very similar to previous preprocessing step but we will remove the rare words from the corpus.

In [ ]:
# Drop the two columns which are no more needed 
df.drop(["text_wo_punct", "text_wo_stop"], axis=1, inplace=True)

n_rare_words = 10
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
def remove_rarewords(text):
    """custom function to remove the rare words"""
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

df["text_wo_stopfreqrare"] = df["text_wo_stopfreq"].apply(lambda text: remove_rarewords(text))

#5.Stemming¶
Stemming is the process of reducing inflected (or sometimes derived) words to their word stem, base or root form (From Wikipedia)

For example, if there are two words in the corpus walks and walking, then stemming will stem the suffix to make them walk. But say in another example, we have two words console and consoling, the stemmer will remove the suffix and make them consol which is not a proper english word.

There are several type of stemming algorithms available and one of the famous one is porter stemmer which is widely used. We can use nltk package for the same.

In [ ]:
from nltk.stem.porter import PorterStemmer

# Drop the two columns 
df.drop(["text_wo_stopfreq", "text_wo_stopfreqrare"], axis=1, inplace=True) 

stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

df["text_stemmed"] = df["text"].apply(lambda text: stem_words(text))
df.head()

We can see that words like private and propose have their e at the end chopped off due to stemming. This is not intented. What can we do fort hat? We can use Lemmatization in such cases.

Also this porter stemmer is for English language. If we are working with other languages, we can use snowball stemmer. The supported languages for snowball stemmer are

In [ ]:
from nltk.stem.snowball import SnowballStemmer
SnowballStemmer.languages

('arabic',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'hungarian',
 'italian',
 'norwegian',
 'porter',
 'portuguese',
 'romanian',
 'russian',
 'spanish',
 'swedish')

#6.Lemmatization
Lemmatization is similar to stemming in reducing inflected words to their word stem but differs in the way that it makes sure the root word (also called as lemma) belongs to the language.

As a result, this one is generally slower than stemming process. So depending on the speed requirement, we can choose to use either stemming or lemmatization.

Let us use the WordNetLemmatizer in nltk to lemmatize our sentences

In [ ]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

df["text_lemmatized"] = df["text"].apply(lambda text: lemmatize_words(text))

###second example

In [ ]:
#instantiate initializer
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
  lem=[lemmatizer.lemmatize(i) for i in text]
  return lem

df['text']=df['text'].apply(lambda x:lemmatize_words(x))

We can see that the trailing e in the propose and private is retained when we use lemmatization unlike stemming.

Wait. There is one more thing in lemmatization. Let us try to lemmatize running now.



In [ ]:

lemmatizer.lemmatize("running")

Wow. It returned running as such without converting it to the root form run. This is because the lemmatization process depends on the POS tag to come up with the correct lemma. Now let us lemmatize again by providing the POS tag for the word.

In [ ]:
lemmatizer.lemmatize("running", "v") # v for verb


Now we are getting the root form run. So we also need to provide the POS tag of the word along with the word for lemmatizer in nltk. Depending on the POS, the lemmatizer may return different results.

Let us take the example, stripes and check the lemma when it is both verb and noun.

In [ ]:
print("Word is : stripes")
print("Lemma result for verb : ",lemmatizer.lemmatize("stripes", 'v'))
print("Lemma result for noun : ",lemmatizer.lemmatize("stripes", 'n'))

Word is : stripes

Lemma result for verb :  strip

Lemma result for noun :  stripe

#Now let us redo the lemmatization process for our dataset.

In [ ]:


from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

df["text_lemmatized"] = df["text"].apply(lambda text: lemmatize_words(text))

We can now see that in the third row, sent got converted to send since we provided the POS tag for lemmatization.

#7.Removal of Emojis¶
With more and more usage of social media platforms, there is an explosion in the usage of emojis in our day to day life as well. Probably we might need to remove these emojis for some of our textual analysis.

Thanks to https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b, please find below a helper function to remove emojis from our text.

In [ ]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

remove_emoji("game is on 🔥🔥")

In [ ]:
remove_emoji("Hilarious😂")

#8.Removal of Emoticons
This is what we did in the last step right? Nope. We did remove emojis in the last step but not emoticons. There is a minor difference between emojis and emoticons.

From Grammarist.com, emoticon is built from keyboard characters that when put together in a certain way represent a facial expression, an emoji is an actual image.

:-) is an emoticon

😀 is an emoji

Thanks to NeelShah for the wonderful collection of emoticons, we are going to use them to remove emoticons.

Please note again that the removal of emojis / emoticons are not always preferred and decision should be made based on the use case at hand.

In [ ]:
def remove_emoticons(text):
    emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
    return emoticon_pattern.sub(r'', text)

remove_emoticons("Hello :-)")

In [ ]:
#ANOTHER EXAMPLE
remove_emoticons("I am sad :(")

#9.Conversion of Emoticon to Words
In the previous step, we have removed the emoticons. In case of use cases like sentiment analysis, the emoticons give some valuable information and so removing them might not be a good solution. What can we do in such cases?

One way is to convert the emoticons to word format so that they can be used in downstream modeling processes. Thanks for Neel again for the wonderful dictionary that we have used in the previous step. We are going to use that again for conversion of emoticons to words.

In [ ]:
def convert_emoticons(text):
    for emot in EMOTICONS:
        text = re.sub(u'('+emot+')', "_".join(EMOTICONS[emot].replace(",","").split()), text)
    return text

text = "Hello :-) :-)"
convert_emoticons(text)

In [ ]:
text = "I am sad :()"
convert_emoticons(text)

###This method might be better for some use cases when we do not want to miss out on the emoticon information.

#10.Conversion of Emoji to Words¶
Now let us do the same for Emojis as well. Neel Shah has put together a list of emojis with the corresponding words as well as part of his https://github.com/NeelShah18/emot/blob/master/emot/emo_unicode.py. We are going to make use of this dictionary to convert the emojis to corresponding words.

Again this conversion might be better than emoji removal for certain use cases. Please use the one that is suitable for the use case.

In [ ]:
def convert_emojis(text):
    for emot in UNICODE_EMO:
        text = re.sub(r'('+emot+')', "_".join(UNICODE_EMO[emot].replace(",","").replace(":","").split()), text)
    return text

text = "game is on 🔥"
convert_emojis(text)

#11.Removal of URLs
Next preprocessing step is to remove any URLs present in the data. For example, if we are doing a twitter analysis, then there is a good chance that the tweet will have some URL in it. Probably we might need to remove them for our further analysis.

We can use the below code snippet to do that.

In [ ]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)
text = "Driverless AI NLP blog post on https://www.h2o.ai/blog/detecting-sarcasm-is-difficult-but-ai-may-have-an-answer/"
remove_urls(text)

###Thanks to Pranjal for the edge cases in the comments below. Suppose say there is no http or https in the url link. The function can now captures that as well.

In [ ]:
text = "Want to know more. Checkout www.h2o.ai for additional information"
remove_urls(text)

#12.Removal of HTML Tags¶
One another common preprocessing technique that will come handy in multiple places is removal of html tags. This is especially useful, if we scrap the data from different websites. We might end up having html strings as part of our text.

First, let us try to remove the HTML tags using regular expressions.

In [ ]:
def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

text = """<div>
<h1> H2O</h1>
<p> AutoML</p>
<a href="https://www.h2o.ai/products/h2o-driverless-ai/"> Driverless AI</a>
</div>"""

print(remove_html(text))

### OUTPUT>>
 H2O

 AutoML

 Driverless AI


We can also use BeautifulSoup package to get the text from HTML document in a more elegant way.

In [ ]:
from bs4 import BeautifulSoup

def remove_html(text):
    return BeautifulSoup(text, "lxml").text

text = """<div>
<h1> H2O</h1>
<p> AutoML</p>
<a href="https://www.h2o.ai/products/h2o-driverless-ai/"> Driverless AI</a>
</div>
"""

print(remove_html(text))


 H2O
 AutoML
 Driverless AI




##Removal of numbers

In [ ]:
def remove_numbers(text):
    # define the pattern to keep
    pattern = r'[^a-zA-z.,!?/:;\"\'\s]' 
    return re.sub(pattern, '', text)

tweet_df["text"] = tweet_df["text"].apply(lambda a:remove_numbers(a) )

#13.Chat Words Conversion
This is an important text preprocessing step if we are dealing with chat data. People do use a lot of abbreviated words in chat and so it might be helpful to expand those words for our analysis purposes.

Got a good list of chat slang words from this repo::https://github.com/rishabhverma17/sms_slang_translator/blob/master/slang.txt. We can use this for our conversion here. We can add more words to this list.

In [ ]:
chat_words_map_dict = {}
chat_words_list = []
for line in chat_words_str.split("\n"):
    if line != "":
        cw = line.split("=")[0]
        cw_expanded = line.split("=")[1]
        chat_words_list.append(cw)
        chat_words_map_dict[cw] = cw_expanded
chat_words_list = set(chat_words_list)

def chat_words_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words_list:
            new_text.append(chat_words_map_dict[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

chat_words_conversion("one minute BRB")

In [ ]:
chat_words_conversion("imo this is awesome")

We can add more words to our abbreviation list and use them based on our use case.

#14.Spelling Correction¶
One another important text preprocessing step is spelling correction. Typos are common in text data and we might want to correct those spelling mistakes before we do our analysis.

If we are interested in wrinting a spell corrector of our own, we can probably start with famous code:https://norvig.com/spell-correct.html from Peter Norvig.

In this notebook, let us use the python package pyspellchecker for our spelling correction.

In [ ]:
from spellchecker import SpellChecker

spell = SpellChecker()
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)
        
text = "speling correctin"
correct_spellings(text)

https://www.geeksforgeeks.org/python-textblob-correct-method/

In [ ]:
#Let's practice spelling and word harmonization
from textblob import TextBlob
tweet_df['text']=tweet_df['text'].apply(lambda x: str(TextBlob(x).correct()))

#15.Spacy text without stop words:
Here you see that nltk and spacy have different vocabulary size, so the results of filtering are different. But the main thing I want to underline is that the word not was filtered, which in most cases will be alright, but in the case when you want to determine the polarity of this sentence not will bring additional meaning.
For such cases, you are able to set stop words you can ignore in spacy library. In the case of nltk you cat just remove or add custom words to nltk_stop_words, it is just a list.

source:https://towardsdatascience.com/text-preprocessing-steps-and-universal-pipeline-94233cb6725a

In [ ]:
import spacy
import en_core_web_sm

nlp = en_core_web_sm.load()
text_without_stop_words = [t.text for t in nlp(text) if not t.is_stop]
display(f"Spacy text without stop words: {text_without_stop_words}")

#16.Removing non_ascii characters

In [ ]:
!pip install unicodedata2
import unicodedata2
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata2.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words
tweet_df["text"] = tweet_df["text"].apply(lambda w:remove_non_ascii(w) )

SOURCE:https://www.kaggle.com/sudalairajkumar/getting-started-with-text-preprocessing

#17.Word Embedding Techniques
There is a huge amount of data in text format. Analyzing text data is an extremely complex task for a machine as it’s difficult for a machine to understand the semantics behind text. At this stage, we’re going to process our text data into a machine-understandable format using word embedding.

    Word Embedding is simply converting data in a text format to numerical values (or vectors) so we can give these vectors  as input to a machine, and analyze the data using the concepts of algebra.
However, it’s important to note that when we perform this transformation there could be data loss. The key then is to maintain an equilibrium between conversion and retaining data.

Here are two commonly used terminologies when it comes to this step.

    Each text data point is called a Document
    An entire set of documents is called a Corpus
Text processing can be done using the following techniques:


    Bag of Words

    TF-IDF

    Word2Vec

Next, let’s explore each of the above techniques in more detail, then decide which to use for our Twitter sentiment analysis model.

##17 (A). Bag of Words¶
Bag of Words does a simple transformation of the document to a vector by using a dictionary of unique words. This is done in just two steps, outlined below.

Construction of Dictionary

Create a dictionary of all the unique words in the data corpus in a vector form. Let the number of unique words in the corpus be, ‘d’. So each word is a dimension and hence this dictionary vector is a d-dimension vector.

Construction of Vectors

For each document, say, rᵢ we create a vector, say, vᵢ.

     This vᵢ which has d-dimensions can be constructed in two ways:
1.For each document, the vᵢ is constructed in accordance with the dictionary such that each word in the dictionary is reproduced by the number of times that word is present in the document.

2.For each document, the vᵢ is constructed in accordance with the dictionary such that each word in the dictionary is reproduced as:

    1 if the word exists in the document or
    0 if the word doesn’t exist in the document
This type is known as a Binary Bag of Words.

Now we have vectors for each document and a dictionary with a set of unique words from the data corpus. These vectors can be analyzed by,

    plotting in d-dimension space or
    calculating distance between vectors to get the similarity (the closer the vectors are, the more similar they are)

#1.WORD2VEC MODEL
In Word2Vec every word is assigned a vector. We start with either a random vector or one-hot vector.

One-Hot vector: A representation where only one bit in a vector is 1.If there are 500 words in the corpus then the vector length will be 500. After assigning vectors to each word we take a window size and iterate through the entire corpus. While we do this there are two neural embedding methods which are used:



Word2Vec consists of models for generating word embedding. These models are shallow two layer neural networks having one input layer, one hidden layer and one output layer. Word2Vec utilizes two architectures :


###-CBOW

 CBOW (Continuous Bag of Words) : CBOW model predicts the current word given context words within specific window. The input layer contains the context words and the output layer contains the current word. The hidden layer contains the number of dimensions in which we want to represent current word present at the output layer.

###-SKIPGRAM
 Skip Gram : Skip gram predicts the surrounding context words within specific window given current word. The input layer contains the current word and the output layer contains the context words. The hidden layer contains the number of dimensions in which we want to represent current word present at the input layer.


#Goal of Word Embeddings

To reduce dimensionality

To use a word to predict the words around it

Inter word semantics must be captured

In [ ]:
import gensim 
from gensim.models import Word2Vec 
# Create CBOW model 
#by default sg=cbow
model1 = gensim.models.Word2Vec(data, min_count = 1, size = 100, window = 5) 


# Create Skip Gram model 
model2 = gensim.models.Word2Vec(data, min_count = 1, size = 100, window = 5, sg = 1) 

#2) GloVe:
This is another method for creating word embeddings. In this method, we take the corpus and iterate through it and get the co-occurence of each word with other words in the corpus. We get a co-occurence matrix through this. The words which occur next to each other get a value of 1, if they are one word apart then 1/2, if two words apart then 1/3 and so on.

#18.Extracting Features from Cleaned Tweets¶
To analyze a preprocessed data, it needs to be converted into features. Depending upon the usage, text features can be constructed using assorted techniques – Bag-of-Words, TF-IDF, and Word Embeddings. In this article, we will be covering only Bag-of-Words and TF-IDF.

#Bag-of-Words Features¶



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
# bag-of-words feature matrix
bow = bow_vectorizer.fit_transform(dataset['Tweet'])

#TF-IDF Features¶

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
# TF-IDF feature matrix
tfidf = tfidf_vectorizer.fit_transform(dataset['Tweet'])

#more on TF-IDF AND BAG OF WORDS

In [ ]:
#Vectorization
#Importing Required Packages

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import gensim

In [ ]:
#Applying Bag of Words Vectorization to the Tweets
bow_vectorizer = CountVectorizer(stop_words= 'english')
bow = bow_vectorizer.fit_transform(dataset['Tweet'])

In [ ]:
#Applying TF-IDF Vectorization to the Tweets
tfidf_vectorizer = TfidfVectorizer(stop_words= 'english')
tfidf = tfidf_vectorizer.fit_transform(dataset['Tweet'])

https://www.geeksforgeeks.org/python-lemmatization-approaches-with-examples/